# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [4]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [5]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [7]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [8]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [10]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [11]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [12]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [15]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [25]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [28]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [34]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [37]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.344007620353042),
 (1, 2.344007620353042),
 (2, 2.344007620353042),
 (3, 2.344007620353042),
 (4, 2.344007620353042)]

## Inverted index

In [40]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [43]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [46]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [50]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('banana', 2), ('what', 10)])]

## TeraSort

In [53]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.01337871801508217),
   (None, 0.02813006723196676),
   (None, 0.10094669704128789),
   (None, 0.15113115794266518),
   (None, 0.16328666878732667),
   (None, 0.18609264137929238),
   (None, 0.18720636894496756),
   (None, 0.20711726851595058),
   (None, 0.24006670409924802),
   (None, 0.26593258976339174),
   (None, 0.26850888173462506),
   (None, 0.31807051941443476),
   (None, 0.34268009597440174),
   (None, 0.3660284110287946),
   (None, 0.3697696219126271),
   (None, 0.41569604512268543),
   (None, 0.4460953522525323),
   (None, 0.4928113902649853),
   (None, 0.49993776567480597)]),
 (1,
  [(None, 0.5032931505464243),
   (None, 0.5675448787944021),
   (None, 0.582879245534588),
   (None, 0.7747839171371524),
   (None, 0.8292982445469145),
   (None, 0.8790818622607203),
   (None, 0.9023174917607603),
   (None, 0.9223413249967685),
   (None, 0.9497322915490252),
   (None, 0.9726829897918838),
   (None, 0.9923484926014428)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [55]:
import random
from functools import reduce

# Функция MAP для нахождения максимального значения в списке чисел
def MAP(numList):
    return max(numList)

# Функция REDUCE для нахождения максимального значения среди всех списков чисел
def REDUCE(max1, max2):
    return max(max1, max2)

# Функция RECORDREADER для генерации случайных чисел в заданном диапазоне
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

# Генерация списка случайных чисел
record = RECORDREADER(15)
print(record)

mapped_values = map(MAP, [record])  
result = reduce(REDUCE, mapped_values)
print("Max value:", result)

[49, 7, 7, 26, 38, 83, 11, 3, 9, 42, 39, 61, 41, 6, 27]
Max value: 83


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [57]:
import random
from typing import Iterator, Tuple

# Функция MAP для создания пары (1, num)
def MAP(num):
    return (1, num)

# Функция REDUCE для вычисления среднего значения из итератора NamedTuple
def REDUCE(key, nums: Iterator[Tuple[int, int]]):
    sum = 0
    count = 0
    for _, n in nums:  # Перебираем только числа, пропуская ключи
        sum += n
        count += 1
    yield sum / count

# Функция RECORDREADER для генерации списка случайных чисел
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

# Генерация списка случайных чисел
record = RECORDREADER(100)

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(MAP, record))
print("MAP output:", map_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(REDUCE(1, map_output))  # Группируем по фиктивному ключу 1

print("Reduce output:", reduce_output[0])

MAP output: [(1, 59), (1, 73), (1, 6), (1, 90), (1, 84), (1, 9), (1, 16), (1, 23), (1, 71), (1, 18), (1, 3), (1, 40), (1, 95), (1, 58), (1, 20), (1, 93), (1, 9), (1, 45), (1, 48), (1, 31), (1, 35), (1, 24), (1, 26), (1, 59), (1, 2), (1, 70), (1, 84), (1, 50), (1, 91), (1, 69), (1, 58), (1, 70), (1, 67), (1, 63), (1, 71), (1, 10), (1, 7), (1, 91), (1, 48), (1, 35), (1, 34), (1, 13), (1, 30), (1, 29), (1, 36), (1, 91), (1, 58), (1, 95), (1, 90), (1, 9), (1, 25), (1, 17), (1, 92), (1, 70), (1, 70), (1, 61), (1, 51), (1, 52), (1, 87), (1, 66), (1, 11), (1, 71), (1, 82), (1, 44), (1, 53), (1, 65), (1, 17), (1, 62), (1, 46), (1, 60), (1, 37), (1, 71), (1, 43), (1, 31), (1, 76), (1, 46), (1, 23), (1, 53), (1, 97), (1, 30), (1, 48), (1, 35), (1, 49), (1, 46), (1, 51), (1, 69), (1, 70), (1, 28), (1, 59), (1, 46), (1, 72), (1, 66), (1, 29), (1, 30), (1, 21), (1, 3), (1, 99), (1, 17), (1, 50), (1, 3)]
Reduce output: 49.06


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [59]:

import random
from typing import Iterator, NamedTuple, List, Tuple

# Функция MAP для создания пары (1, num)
def MAP(num):
    return (1, num)

# Функция REDUCE для вычисления среднего значения из итератора NamedTuple
def REDUCE(key, nums: Iterator[int]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

# Функция RECORDREADER для генерации списка случайных чисел
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

# Функция для группировки значений по ключу с использованием сортировки
def group_by_key(iterable: List[Tuple[int, int]]):
    grouped_data = {}
    for key, value in sorted(iterable, key=lambda x: x[0]):
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP output:", map_output)

# Группировка по ключу и преобразование в список
shuffle_output = group_by_key(map_output)
print("Shuffle output:", shuffle_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print("Reduce output:", reduce_output)
     

MAP output: [(1, 19), (1, 54), (1, 43), (1, 71), (1, 47), (1, 99), (1, 45), (1, 74), (1, 3), (1, 42), (1, 32), (1, 62), (1, 9), (1, 96), (1, 57), (1, 36), (1, 33), (1, 17), (1, 78), (1, 1), (1, 44), (1, 83), (1, 2), (1, 20), (1, 9), (1, 78), (1, 21), (1, 89), (1, 45), (1, 25), (1, 31), (1, 42), (1, 92), (1, 92), (1, 50), (1, 15), (1, 75), (1, 4), (1, 10), (1, 0), (1, 77), (1, 28), (1, 98), (1, 4), (1, 25), (1, 55), (1, 50), (1, 66), (1, 11), (1, 18), (1, 3), (1, 35), (1, 47), (1, 54), (1, 85), (1, 85), (1, 8), (1, 90), (1, 7), (1, 49), (1, 73), (1, 44), (1, 72), (1, 66), (1, 61), (1, 26), (1, 96), (1, 0), (1, 21), (1, 94), (1, 99), (1, 9), (1, 60), (1, 95), (1, 95), (1, 78), (1, 52), (1, 33), (1, 76), (1, 40), (1, 43), (1, 91), (1, 48), (1, 24), (1, 61), (1, 42), (1, 59), (1, 68), (1, 82), (1, 12), (1, 45), (1, 73), (1, 54), (1, 84), (1, 16), (1, 98), (1, 30), (1, 8), (1, 60), (1, 48)]
Shuffle output: [(1, [19, 54, 43, 71, 47, 99, 45, 74, 3, 42, 32, 62, 9, 96, 57, 36, 33, 17, 78, 1, 44

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [136]:
from typing import Iterator
import numpy as np


def flatten(nested_iterable):
    # Функция для "выравнивания" вложенной структуры данных
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    # Группировка данных по ключу
    grouped_data = {}
    for key, value in iterable:
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

def groupbykey_distributed(map_partitions, PARTITIONER):
    # Распределенная группировка данных по ключу
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    # Функция для определения разделителя (партиционера)
    global reducers
    return hash(obj) % reducers

# Функция для выполнения операций MapReduce
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):

    # Применяем функцию MAP к каждой записи во входных данных, затем "выравниваем" результаты
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    # Если задан комбайнер (COMBINER), применяем его к данным после группировки по ключу
    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    # Выполняем распределенную группировку данных по ключу с использованием заданного партиционера
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    # Применяем функцию REDUCE к каждой группе данных после группировки и "выравниваем" результаты
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group),
                                                                                    reduce_partition[1]))), reduce_partitions)
       # Вычисляем общее количество отправленных по сети пар ключ-значение
    total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])

    # Выводим информацию о количестве отправленных данных и возвращаем результат операций MapReduce
    print(f"{total_values_sent} key-value pairs were sent over a network.")
    return reduce_outputs

# Задание текстовых документов
d1 = """
fast cat runs
runs cat fast
cat runs fast"""

d2 = """
runs fast
fast cat
cat runs"""

d3 = """
dog and cat run together"""
documents = [d1, d2, d3]


maps = 3
reducers = 2

def INPUTFORMAT():
    # Форматирование входных данных для MapReduce
    global maps

    # Функция RECORDREADER принимает на вход раздел (split) данных и генерирует ключ-значение для каждой строки документа
    def RECORDREADER(split):
      # Перебираем каждый документ в разделе, присваивая ему уникальный идентификатор docid
        for (docid, document) in enumerate(split):
          # Для каждой строки в документе создаем уникальный идентификатор lineid и разбиваем строку по символу новой строки '\n'
            for (lineid, line) in enumerate(document.split('\n')):
               # Генерируем ключ в формате "docid:lineid" и значение - строку из документа
                yield ("{}:{}".format(docid, lineid), line)
    # Вычисляем размер раздела данных split_size как округленное вверх
    #значение от деления общего числа документов на количество задач MapReduce
    split_size = int(np.ceil(len(documents) / maps))
    # Цикл по документам с шагом split_size
    for i in range(0, len(documents), split_size):
      # Генерируем данные для каждого раздела данных путем вызова функции RECORDREADER с подмножеством документов
        yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str):
    # Функция Map для преобразования данных
    for word in line.split(" "):
        yield (word, word)

def REDUCE(key: str, value: Iterator[str]):
    # Функция Reduce для агрегации данных
    yield key

# Выполнение MapReduce операций с заданными функциями MAP и REDUCE
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

23 key-value pairs were sent over a network.


[(0, ['', 'and', 'cat', 'fast', 'run', 'together']), (1, ['dog', 'runs'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [120]:
from collections import defaultdict
import random

# Функция для реализации MAP операции
def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C - выбираем только кортежи, удовлетворяющие условию
            mapped_result[t].append(t)
    return mapped_result.items()

# Функция для реализации REDUCE операции
def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)  # Выводим промежуточный результат
    for values in mapped_items:
        for value in values:
            reduced_result.append(value)  # Функция идентичности: возвращаем то же значение, что получили на вход
    return reduced_result

# Предикат для фильтрации кортежей с четным первым элементом
def C(t):
    return t[0] % 2 == 0

# Генерация случайных записей (кортежей формата (x, y, z))
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

# Создание набора записей
record = RECORDREADER(5)
print(record)

# Разбиение записей на равные части
part_count = 5
record_partitioned = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitioned)

# Применение операции MAP и последующей REDUCE к разделенным записям
print(REDUCE(list(map(lambda x: MAP(x), record_partitioned))))

[(28, 89, 82), (89, 87, 82), (61, 16, 53), (45, 84, 22), (21, 10, 25)]
[[(28, 89, 82), (89, 87, 82), (61, 16, 53), (45, 84, 22), (21, 10, 25)]]
[dict_items([((28, 89, 82), [(28, 89, 82)])])]
[((28, 89, 82), [(28, 89, 82)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [122]:
S = set([25, 90, 69])  # Создаем множество S

def MAP(t):
    res_list = []
    for el in t:  # Перебираем элементы входного кортежа t
        if el in S:  # Проверяем, есть ли элемент el в множестве S
            res_list.append(el)  # Добавляем элемент el в результирующий список
    res = tuple(res_list)  # Преобразуем список в кортеж
    return (res, res)  # Возвращаем кортеж из результирующего списка дважды

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(5)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(100)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список

print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем функцию REDUCE к результату группировки
print("Reduce output:", reduce_output)

[(4, 14, 10), (16, 52, 74), (71, 47, 42), (38, 44, 61), (15, 35, 5)]
MAP output: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((90, 25), (90, 25)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((90,), (90,)), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((),

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [67]:
def MAP(t):
    return (t, t)  # Функция MAP возвращает кортеж из входного элемента и этого же элемента

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Функция REDUCE возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(5)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(100)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем функцию REDUCE к результату группировки
print("Reduce output:", reduce_output)

[(31, 0, 60), (73, 87, 69), (51, 8, 37), (44, 41, 15), (70, 54, 57)]
MAP output: [((94, 11, 27), (94, 11, 27)), ((26, 61, 41), (26, 61, 41)), ((36, 81, 49), (36, 81, 49)), ((63, 24, 86), (63, 24, 86)), ((12, 99, 48), (12, 99, 48)), ((39, 31, 50), (39, 31, 50)), ((71, 45, 97), (71, 45, 97)), ((57, 95, 9), (57, 95, 9)), ((60, 79, 73), (60, 79, 73)), ((79, 42, 15), (79, 42, 15)), ((25, 51, 5), (25, 51, 5)), ((2, 54, 65), (2, 54, 65)), ((4, 47, 66), (4, 47, 66)), ((26, 91, 43), (26, 91, 43)), ((16, 70, 65), (16, 70, 65)), ((43, 24, 11), (43, 24, 11)), ((37, 20, 97), (37, 20, 97)), ((4, 93, 65), (4, 93, 65)), ((32, 4, 95), (32, 4, 95)), ((97, 97, 0), (97, 97, 0)), ((79, 50, 18), (79, 50, 18)), ((11, 16, 75), (11, 16, 75)), ((68, 63, 31), (68, 63, 31)), ((25, 2, 91), (25, 2, 91)), ((11, 38, 22), (11, 38, 22)), ((31, 83, 2), (31, 83, 2)), ((72, 71, 98), (72, 71, 98)), ((53, 90, 67), (53, 90, 67)), ((0, 11, 66), (0, 11, 66)), ((98, 97, 39), (98, 97, 39)), ((66, 98, 11), (66, 98, 11)), ((2, 51,

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [69]:

def MAP(t):
    return (t, t)  # Функция MAP возвращает кортеж из входного элемента и этого же элемента

def REDUCE(key, values: Iterator[NamedTuple]):
    if len(values) == 2:  # Если количество значений равно 2
        return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(100)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(10)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список
print("Shuffle output:", shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]  # Применяем функцию REDUCE к результату группировки
print("Reduce output:", reduce_output)

[(3, 2), (2, 0), (2, 2), (3, 1), (0, 0), (1, 1), (0, 2), (2, 3), (0, 1), (0, 1), (3, 1), (0, 0), (3, 2), (2, 0), (1, 0), (1, 1), (0, 0), (2, 2), (0, 3), (3, 3), (1, 0), (3, 0), (1, 0), (3, 0), (3, 3), (0, 2), (1, 3), (0, 2), (0, 3), (1, 3), (1, 0), (3, 0), (3, 0), (0, 3), (0, 3), (3, 0), (3, 3), (2, 3), (2, 0), (0, 1), (1, 0), (1, 1), (3, 0), (3, 1), (1, 3), (1, 3), (1, 2), (0, 2), (2, 2), (2, 0), (1, 3), (3, 3), (2, 2), (0, 1), (0, 0), (1, 0), (2, 2), (1, 3), (1, 1), (2, 0), (2, 1), (3, 2), (1, 2), (3, 0), (2, 2), (1, 3), (2, 2), (3, 1), (2, 3), (0, 0), (3, 3), (2, 2), (3, 2), (1, 2), (2, 2), (0, 1), (3, 3), (2, 3), (0, 3), (3, 2), (2, 1), (3, 3), (0, 3), (1, 1), (0, 2), (3, 2), (2, 1), (3, 3), (3, 1), (0, 1), (3, 3), (1, 0), (3, 3), (1, 3), (0, 2), (3, 3), (3, 1), (3, 1), (0, 2), (2, 1)]
MAP output: [((0, 3), (0, 3)), ((3, 3), (3, 3)), ((0, 3), (0, 3)), ((0, 3), (0, 3)), ((1, 2), (1, 2)), ((2, 2), (2, 2)), ((0, 1), (0, 1)), ((1, 1), (1, 1)), ((1, 1), (1, 1)), ((3, 3), (3, 3)), ((1, 2

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [71]:
rels = [1, 2]  # Инициализация массива rels с элементами 1 и 2

class Tuple:  # Определение класса Tuple для хранения данных и id представления
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])  # Генерация случайного кортежа данных
    rel_id = rels[random.randint(0, len(rels) - 1)]  # Выбор случайного id представления из массива rels
    return Tuple(data, rel_id)  # Возвращает экземпляр класса Tuple с данными и id представления

def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]  # Генерация списка случайных кортежей данных

def MAP(t: Tuple):
    return (t.data, t.rel_id)  # Функция MAP возвращает кортеж из данных и id представления

def REDUCE(key, values: Iterator[NamedTuple]):
    if values == [rels[0]]:  # Если значение соответствует первому элементу в массиве rels
        return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]  # Группировка значений по ключу k2
    return t.items()  # Возвращает список кортежей (ключ, список значений)

record = RECORDREADER(100)  # Генерация списка случайных кортежей данных

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу из RECORDREADER(100)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка значений по ключу
shuffle_output = list(shuffle_output)  # Преобразование результата в список
print("Shuffle output:", shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]  # Применение функции REDUCE к результату группировки
print("Reduce output:", reduce_output)
     

MAP output: [(((3, 3), (1, 0), (3, 2)), 1), (((0, 1), (3, 2), (2, 0)), 2), (((2, 3), (1, 0), (3, 2)), 2), (((1, 1), (1, 3), (3, 1)), 2), (((3, 1), (0, 0), (0, 2)), 1), (((3, 3), (0, 0), (0, 2)), 1), (((1, 3), (3, 2), (1, 2)), 2), (((2, 0), (2, 2), (0, 3)), 1), (((2, 2), (2, 3), (3, 3)), 1), (((2, 3), (1, 3), (2, 1)), 1), (((3, 3), (2, 0), (3, 1)), 1), (((1, 1), (3, 0), (0, 2)), 1), (((3, 2), (1, 1), (1, 2)), 2), (((1, 3), (0, 2), (3, 1)), 2), (((0, 2), (2, 2), (1, 1)), 1), (((1, 3), (3, 0), (2, 2)), 2), (((2, 1), (0, 2), (1, 2)), 2), (((1, 3), (2, 3), (1, 3)), 1), (((2, 1), (1, 3), (0, 1)), 2), (((0, 3), (0, 2), (0, 1)), 2), (((3, 0), (2, 0), (0, 3)), 2), (((1, 2), (2, 2), (2, 2)), 2), (((2, 1), (0, 2), (1, 2)), 1), (((3, 3), (3, 1), (3, 1)), 1), (((1, 1), (2, 2), (1, 1)), 1), (((3, 2), (2, 0), (3, 3)), 2), (((2, 3), (3, 1), (0, 1)), 2), (((3, 1), (2, 3), (0, 2)), 2), (((0, 1), (0, 3), (3, 1)), 1), (((1, 1), (1, 0), (0, 3)), 2), (((0, 1), (0, 0), (1, 2)), 2), (((0, 2), (3, 1), (1, 1)),

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [73]:
rels = [1, 2]  # Инициализация списка идентификаторов связей

class Tuple:  # Определение класса кортежа, хранящего данные и идентификатор связи из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():  # Функция для генерации случайного кортежа с данными и идентификатором связи
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):  # Функция для преобразования данных кортежа в соответствии с условием
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для объединения данных по ключу
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):  # Функция для группировки данных по ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)  # Создание списка случайных кортежей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print("Reduce output:", reduce_output)

MAP output: [(2, (1, 3)), (3, (1, 2)), (2, (2, 2)), (3, (1, 3)), (3, (2, 0)), (2, (1, 2)), (0, (1, 2)), (1, (1, 2)), (0, (2, 3)), (2, (1, 0)), (0, (2, 3)), (0, (1, 2)), (1, (2, 1)), (0, (1, 2)), (1, (2, 1)), (1, (2, 1)), (0, (2, 3)), (2, (2, 3)), (3, (2, 1)), (0, (2, 3)), (2, (1, 0)), (1, (2, 3)), (2, (2, 2)), (3, (1, 0)), (3, (1, 1)), (1, (2, 2)), (0, (2, 1)), (0, (2, 3)), (1, (2, 2)), (3, (1, 3)), (0, (1, 0)), (2, (1, 1)), (0, (1, 2)), (3, (2, 1)), (0, (1, 0)), (3, (1, 3)), (1, (2, 0)), (0, (1, 0)), (2, (2, 0)), (0, (2, 0)), (1, (2, 0)), (3, (1, 0)), (1, (1, 3)), (3, (2, 2)), (1, (2, 3)), (3, (1, 1)), (3, (2, 1)), (0, (2, 0)), (1, (1, 0)), (0, (1, 2)), (2, (2, 2)), (0, (1, 3)), (2, (2, 1)), (0, (1, 1)), (0, (2, 3)), (3, (2, 2)), (2, (2, 2)), (1, (1, 3)), (3, (2, 2)), (0, (1, 2)), (2, (2, 0)), (1, (1, 1)), (3, (1, 1)), (0, (2, 3)), (3, (2, 1)), (3, (1, 0)), (0, (2, 0)), (0, (2, 1)), (2, (1, 2)), (2, (2, 3)), (2, (1, 2)), (0, (1, 0)), (1, (2, 0)), (0, (2, 0)), (1, (1, 3)), (1, (1, 0)),

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [87]:
def get_random_tuple():  # Функция для генерации случайного кортежа с тремя случайными значениями от 0 до 3
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):  # Функция для преобразования входного кортежа, возвращающая первые два элемента кортежа
  return (t[0], t[1])

def tetta(values):  # Функция для вычисления суммы значений в переданном списке
  return sum(values)

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для вычисления суммы значений в переданных значениях и возврата пары (ключ, сумма)
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):  # Функция для группировки данных по ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)  # Создание списка случайных кортежей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print("Reduce output:", reduce_output)
     

MAP output: [(0, 1), (1, 2), (1, 2), (0, 1), (2, 3), (0, 1), (2, 1), (0, 0), (3, 1), (3, 0), (2, 0), (2, 3), (1, 0), (1, 0), (0, 2), (0, 0), (2, 1), (2, 3), (0, 2), (3, 3), (0, 3), (0, 2), (3, 0), (2, 3), (3, 0), (3, 0), (2, 2), (0, 1), (1, 3), (3, 2), (2, 0), (0, 0), (1, 2), (2, 3), (1, 1), (0, 1), (0, 0), (2, 3), (1, 2), (2, 0), (3, 1), (1, 1), (0, 0), (0, 1), (2, 1), (0, 3), (1, 2), (0, 1), (3, 0), (2, 1), (1, 3), (2, 1), (3, 3), (2, 3), (1, 1), (2, 2), (2, 2), (1, 3), (2, 3), (2, 1), (3, 3), (0, 2), (1, 3), (1, 0), (1, 3), (0, 3), (0, 2), (3, 3), (1, 3), (0, 3), (0, 2), (3, 1), (1, 3), (0, 3), (0, 2), (0, 3), (2, 2), (1, 2), (1, 1), (0, 3), (3, 1), (1, 0), (1, 1), (0, 1), (3, 2), (3, 3), (2, 0), (3, 0), (2, 2), (0, 2), (1, 3), (2, 1), (1, 1), (2, 1), (1, 1), (0, 3), (1, 3), (1, 2), (2, 1), (2, 0)]
Shuffle output: [(0, [1, 1, 1, 0, 2, 0, 2, 3, 2, 1, 0, 1, 0, 0, 1, 3, 1, 2, 3, 2, 3, 2, 3, 2, 3, 3, 1, 2, 3]), (1, [2, 2, 0, 0, 3, 2, 1, 2, 1, 2, 3, 1, 3, 3, 0, 3, 3, 3, 2, 1, 0, 1, 3, 1,

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [93]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [101]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [103]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [105]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [108]:
import numpy as np  # Импорт библиотеки numpy под псевдонимом np
I = 2  # Задание значения переменной I равным 2
J = 3  # Задание значения переменной J равным 3
K = 4*10  # Задание значения переменной K равным 40
small_mat = np.random.rand(I,J)  # Создание случайной матрицы размером IxJ
big_mat = np.random.rand(J,K)  # Создание случайной матрицы размером JxK
reference_solution = np.matmul(small_mat, big_mat)  # Вычисление матричного произведения small_mat и big_mat

def RECORDREADER():  # Определение функции RECORDREADER без аргументов
  for i in range(small_mat.shape[0]):  # Цикл по строкам матрицы small_mat
    for j in range(small_mat.shape[1]):  # Цикл по столбцам матрицы small_mat
      yield ((0, i, j), small_mat[i,j])  # Генерация кортежа для первой матрицы

  for j in range(big_mat.shape[0]):  # Цикл по строкам матрицы big_mat
    for k in range(big_mat.shape[1]):  # Цикл по столбцам матрицы big_mat
      yield ((1, j, k), big_mat[j,k])  # Генерация кортежа для второй матрицы

def MAP_JOIN(k1, v1):  # Определение функции MAP_JOIN с ключем k1 и значением v1
  (mat_num, i, j) = k1  # Распаковка ключа
  w = v1  # Присвоение значения из значения
  if mat_num == 0:  # Если номер матрицы равен 0
    yield (j, (mat_num, i, w))  # Генерация значения для первой матрицы
  else:  # Иначе
    yield (i, (mat_num, j, w))  # Генерация значения для второй матрицы

def REDUCE_JOIN(key, values):  # Определение функции REDUCE_JOIN с ключем key и значениями values
  from_first_mat = [v for v in values if v[0] == 0]  # Фильтрация значений для первой матрицы
  from_second_mat = [v for v in values if v[0] == 1]  # Фильтрация значений для второй матрицы
  for f in from_first_mat:  # Цикл по элементам из первой матрицы
    for s in from_second_mat:  # Цикл по элементам из второй матрицы
      yield ((f[1], s[1]), f[2] * s[2])  # Генерация умноженных значений


def MAP_MUL(k1, v1):  # Определение функции MAP_MUL с ключем k1 и значением v1
  (i, k) = k1  # Распаковка ключа
  yield ((i, k), v1)  # Генерация кортежа с ключом (i, k) и значением v1

def REDUCE_MUL(key, values):  # Определение функции REDUCE_MUL с ключем key и значениями values
  res_el_value = 0  # Инициализация переменной для хранения результата
  for v in values:  # Цикл по значениям
    res_el_value += v  # Суммирование значений
  yield (key, res_el_value)  # Генерация кортежа с ключом key и суммой значений

def GET_JOINED():  # Определение функции GET_JOINED без аргументов
  for j in joined:  # Цикл по элементам из joined
    yield j  # Генерация элемента


joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)  # Применение MapReduce к функциям RECORDREADER, MAP_JOIN и REDUCE_JOIN
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)  # Применение MapReduce к функциям GET_JOINED, MAP_MUL и REDUCE_MUL
np.allclose(reference_solution, asmatrix(solution))  # Проверка на равенство решения с эталонным значением и вывод результата (True/False)
     

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [110]:
# Определение размеров матриц
I = 2
J = 3
K = 4*10

# Генерация случайных матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  # Разделение данных на части и применение MAP функции
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  # Применение COMBINER (если есть)
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  # Распределенная группировка данных по ключу (shuffle)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  # Применение REDUCE функции
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Преобразование результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

# Проверка результата на соответствие эталонному решению
np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.32472945756484656), ((0, 1), 0.39061349772243725), ((0, 2), 0.07615572771532945), ((0, 3), 0.1629251780362513), ((0, 4), 0.3708106391514218), ((0, 5), 0.043291382412708045), ((0, 6), 0.2948744717790148), ((0, 7), 0.454550408805967), ((0, 8), 0.2759143471404575), ((0, 9), 0.3479441164957003), ((0, 10), 0.11460047862666807), ((0, 11), 0.2220118247688503), ((0, 12), 0.48094869079084135), ((0, 13), 0.47669230181825345), ((0, 14), 0.17741511924493564), ((0, 15), 0.3158729655192467), ((0, 16), 0.01654147778453637), ((0, 17), 0.47989558523863074), ((0, 18), 0.08442645069719935), ((0, 19), 0.3999486384790662), ((0, 20), 0.25788330908229723), ((0, 21), 0.03638964034276429), ((0, 22), 0.03350947903068666), ((0, 23), 0.002879893755389733), ((0, 24), 0.2260087536096375), ((0, 25), 0.21925618651184503), ((0, 26), 0.24883948326225588), ((0, 27), 0.23424842953958203), ((0, 28), 0.10510645521808473), ((0, 29), 0.17736719554266772), ((0, 30

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [112]:
# Определение размеров матриц
I = 2  # Количество строк первой матрицы и строк в итоговой матрице
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4*10  # Количество столбцов второй матрицы и столбцов в итоговой матрице

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Генерация первой матрицы
big_mat = np.random.rand(J, K)  # Генерация второй матрицы

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.6865604225085846), ((0, 1), 0.4988502323948747), ((0, 2), 0.9051808574381399), ((0, 3), 0.11548222467679138), ((0, 4), 0.4285204028690292), ((0, 5), 0.6458516068629642), ((0, 6), 0.6959052660213096), ((0, 7), 0.8888632257325753), ((0, 8), 0.622940582746046), ((0, 9), 0.43731335010344835), ((0, 10), 0.7139779366512209), ((0, 11), 0.06218031446509917), ((0, 12), 0.25576192343846854), ((0, 13), 0.9214804792471838), ((0, 14), 0.40958310847029983), ((0, 15), 0.8451289201217963), ((0, 16), 0.5107325823797799), ((0, 17), 0.08750643607871707), ((0, 18), 0.7756328383736067), ((0, 19), 0.5300475012477469), ((0, 20), 0.41440786644603445), ((0, 21), 0.8031992814803026), ((0, 22), 0.7365733144313312), ((0, 23), 0.9360955408272155), ((0, 24), 0.08942090266342465), ((0, 25), 0.4129056300769295), ((0, 26), 0.8523873212481062), ((0, 27), 0.6008273655245934), ((0, 28), 0.9136075212812913), ((0, 29), 0.14184038287369508), ((0, 30), 0.31297030

True